## Anton Anisimov NLP HM №3

In [10]:
import pandas as pd
import numpy as np
# data prep
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup
import warnings

from sklearn.ensemble import RandomForestClassifier
import nltk
from nltk.corpus import stopwords # Import the stop word list
from gensim.models import word2vec
# models
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [2]:
warnings.filterwarnings("ignore")

In [33]:
# load tran set
df_train=pd.read_csv("train.tsv", sep="\t",quoting=3)
print ('Train set balance')
print (df_train['Sentiment'].value_counts())
# load test set
df_test=pd.read_csv("test.tsv", sep="\t",quoting=3)
df_y=pd.read_csv("sampleSubmission.csv")
df_test['Sentiment']=df_y['Sentiment']
print ('Test set balance')
print (df_test['Sentiment'].value_counts())
# Merge sets
a=df_train['Sentiment'].tolist()+df_test['Sentiment'].tolist()
df=pd.concat([df_train, df_test], axis=0,sort=False)
df.index= df['PhraseId']
df=df.drop(columns=['PhraseId','SentenceId'])
df['Sentiment']=a
print (df.shape)
df.head()

Train set balance
2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64
Test set balance
2    66292
Name: Sentiment, dtype: int64
(222352, 2)


,Phrase,Sentiment
PhraseId,,
1,A series of escapades demonstrating the adage ...,1
2,A series of escapades demonstrating the adage ...,2
3,A series,2
4,A,2
5,series,2


As we can see above test set consists of class 2.So for testing perfomance I decided to choose f1 weighted metric

## Pipeline:</br>
    1)Merge sets to create dictionaries
    2)Create 3 datasets(bow,tfidf,w2v)
    3)Split to train and test
    4)Train and test two models (log-reg and random forest)
    6)Try to tune and balance
    7)Compare results

Now, remember the last index of train set.After clean merged set and create bag and ifidf. Finally split to train and test by rembered indexes 

In [10]:
split_ind=df_train['PhraseId'].max()

In [11]:
labels=df['Sentiment'].tolist()

## CLEAN DATA

In [12]:
# primary cleaning
def review_to_words( raw_review ):
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
     # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    return( " ".join( meaningful_words )) 

In [13]:
# delete trash1 
num_reviews = df["Phrase"].size
clean_train_reviews = []
for i in range(0, num_reviews):
    clean_train_reviews.append( review_to_words( df.iloc[i,0] ) )

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this 

In [2]:
#reconstract table
a=df.index
df=pd.DataFrame(clean_train_reviews,columns=['Obs'])
df['features']=labels
df.index=a
df.head()

NameError: name 'df' is not defined

In [17]:
# delete empty raws
df.replace('', np.nan, inplace=True)
df.dropna(subset=['Obs'], inplace=True)
# delete duplicates
df.drop_duplicates(subset ='Obs', keep = False, inplace = True)

raw data set=156060,
new preclean data =40448
special ind=156060

In [7]:
df.to_csv('data_merge_clean.csv')

## Load check point data

In [4]:
split_ind=156060

In [8]:
# load data function just for convince
def load_set(name):
    df=pd.read_csv(name)
    df.index=df.iloc[:,0]
    df=df.drop(columns=df.columns[0])
    return df

In [6]:
df=load_set('data_merge_clean.csv')
df.head()

,Obs,features
PhraseId,,
1,series escapades demonstrating adage good goos...,1
2,series escapades demonstrating adage good goose,2
9,escapades,2
10,demonstrating adage good goose,2
11,demonstrating adage,2


In [7]:
df['features'].value_counts()

2    33145
3     7750
1     6670
4     2120
0     1620
Name: features, dtype: int64

Сlasses are unbalased, so<br/> 1)ignore<br/> 2)naive cut <br/> 3)custom penalty in lost function<br/> 

## Words vectorization

Bow:

In [8]:
# bags transform
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None) 
train_data_features = vectorizer.fit_transform(df['Obs'],df.iloc[:,-1])
colums=vectorizer.get_feature_names()
train_data_features = train_data_features.toarray()
df_bags=pd.DataFrame(train_data_features,columns=colums,index=df.index)
df_bags['f_labels']=df.iloc[:,-1].tolist()
df_bags.index=df.index
# df_bags.to_csv('bow.csv')
df_bags.shape

(51305, 17188)

Becuse of memory leak I will use models consistently:

Log-reg BOW:

In [9]:
score_log=[]

In [10]:
# my log for conv
def my_log(X_train,X_test,y_train,y_test):
    Model = LogisticRegression(random_state=0, solver='newton-cg',
                           multi_class='multinomial',penalty='l2').fit(X_train, y_train)
    y_pred=Model.predict(X_train)
    print (metrics.classification_report(y_train,y_pred))
    train_scores=metrics.f1_score(y_train,y_pred,average='macro')
    train_scores_1=metrics.f1_score(y_train,y_pred,average='weighted')
    y_pred=Model.predict(X_test)
    test_scores=metrics.f1_score(y_test,y_pred,average='weighted')
    print ('train_macro:',train_scores,'train_weighted:',train_scores_1,'test:',test_scores)
    return Model,y_pred

In [11]:
# Data split
index_train=df.index<=split_ind
a=(index_train+0).sum()
# bow split
X_b_train, X_b_test =df_bags.iloc[:a],df_bags.iloc[a:]
del df_bags

In [12]:
# run log reg
_,y_pred=my_log(X_b_train.iloc[:,:-1],X_b_test.iloc[:,:-1],X_b_train.iloc[:,-1],X_b_test.iloc[:,-1])    

              precision    recall  f1-score   support

           0       0.89      0.52      0.65      1620
           1       0.79      0.58      0.67      6670
           2       0.75      0.95      0.84     18924
           3       0.79      0.63      0.70      7750
           4       0.89      0.59      0.71      2120

    accuracy                           0.77     37084
   macro avg       0.82      0.65      0.71     37084
weighted avg       0.78      0.77      0.76     37084

train_macro: 0.7148329826287129 train_weighted: 0.764255165847601 test: 0.8128886848365959


In [13]:
print (metrics.classification_report(X_b_test.iloc[:,-1],y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.68      0.81     14221
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

    accuracy                           0.68     14221
   macro avg       0.20      0.14      0.16     14221
weighted avg       1.00      0.68      0.81     14221



So train score are bad, lets try to use stupid cut(to 2000):

In [14]:
# check balance
X_b_train['f_labels'].value_counts()

2    18924
3     7750
1     6670
4     2120
0     1620
Name: f_labels, dtype: int64

In [15]:
# balance cut
for i in range(1,5):
    a=X_b_train['f_labels']==i
    a=X_b_train.index[a]
    shuffle_ap=np.random.permutation(len(a))
    index=a[shuffle_ap[3000:]]
    if i == 1:
        X_b_train1=X_b_train.drop(index=index)
    else:
        X_b_train1=X_b_train1.drop(index=index)

In [16]:
X_b_train1['f_labels'].value_counts()

3    3000
2    3000
1    3000
4    2120
0    1620
Name: f_labels, dtype: int64

In [17]:
_,y_pred=my_log(X_b_train1.iloc[:,:-1],X_b_test.iloc[:,:-1],X_b_train1.iloc[:,-1],X_b_test.iloc[:,-1]) 

              precision    recall  f1-score   support

           0       0.93      0.81      0.87      1620
           1       0.86      0.89      0.88      3000
           2       0.84      0.91      0.87      3000
           3       0.87      0.86      0.86      3000
           4       0.90      0.86      0.88      2120

    accuracy                           0.87     12740
   macro avg       0.88      0.87      0.87     12740
weighted avg       0.87      0.87      0.87     12740

train_macro: 0.8712383644151173 train_weighted: 0.8712833741630417 test: 0.6295653849860268


In [18]:
print (metrics.classification_report(X_b_test.iloc[:,-1],y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.46      0.63     14221
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

    accuracy                           0.46     14221
   macro avg       0.20      0.09      0.13     14221
weighted avg       1.00      0.46      0.63     14221



Result:maybe it was a bad idea,but this model now provides with balanced prediction. Moreover on test set we lost 0.23 on recall, but precision still be 1.

Random forest BOW:

I made several experiments with cv and grid serch and found some optimal parametrs

In [19]:
def my_rf(X_train,X_test,y_train,y_test,nt,deep):
    Model = RandomForestClassifier(n_estimators=nt,max_depth=deep).fit(X_train, y_train)
    y_pred=Model.predict(X_train)
    train_scores=metrics.f1_score(y_train,y_pred,average='macro')
    train_scores_1=metrics.f1_score(y_train,y_pred,average='weighted')
    y_pred=Model.predict(X_test)
    test_scores=metrics.f1_score(y_test,y_pred,average='weighted')
    print ('train_macro:',train_scores,'train_weighted:',train_scores_1,'test:',test_scores)
    return Model,y_pred

In [20]:
my_rf(X_b_train.iloc[:,:-1],X_b_test.iloc[:,:-1],X_b_train.iloc[:,-1],X_b_test.iloc[:,-1],150,70)   

train_macro: 0.3894973771800297 train_weighted: 0.5059661768980499 test: 0.9894475039971575


(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                        max_depth=70, max_features='auto', max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=150,
                        n_jobs=None, oob_score=False, random_state=None,
                        verbose=0, warm_start=False),
 array([2, 2, 2, ..., 2, 2, 2], dtype=int64))

Tfidf:

In [21]:
vectorizer = TfidfVectorizer()
Vec=vectorizer.fit_transform(df['Obs'],df.iloc[:,-1])
col=vectorizer.get_feature_names()
df_tf=pd.DataFrame(Vec.todense(),columns=col)
df_tf['f_labels']=df.iloc[:,-1].tolist()
df_tf.index=df.index
# df_tf.to_csv('tfidf.csv')
df_tf.shape

(51305, 17188)

In [22]:
# Data split
index_train=df.index<=split_ind
a=(index_train+0).sum()
# bow split
X_tf_train, X_tf_test =df_tf.iloc[:a],df_tf.iloc[a:]
del df_tf

Log-reg tfidf:

In [23]:
_,y_pred=my_log(X_tf_train.iloc[:,:-1],X_tf_test.iloc[:,:-1],X_tf_train.iloc[:,-1],X_tf_test.iloc[:,-1])  

              precision    recall  f1-score   support

           0       0.81      0.16      0.26      1620
           1       0.70      0.42      0.53      6670
           2       0.68      0.94      0.79     18924
           3       0.70      0.53      0.61      7750
           4       0.83      0.27      0.41      2120

    accuracy                           0.69     37084
   macro avg       0.74      0.47      0.52     37084
weighted avg       0.70      0.69      0.66     37084

train_macro: 0.5194305465404697 train_weighted: 0.6601275296344653 test: 0.8674842717209524


In [24]:
print (metrics.classification_report(X_tf_test.iloc[:,-1],y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.77      0.87     14221
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

    accuracy                           0.77     14221
   macro avg       0.20      0.15      0.17     14221
weighted avg       1.00      0.77      0.87     14221



RF tfidf:

In [25]:
_,y_pred=my_rf(X_tf_train.iloc[:,:-1],X_tf_test.iloc[:,:-1],X_tf_train.iloc[:,-1],X_tf_test.iloc[:,-1],250,70)  

train_macro: 0.3745370902802025 train_weighted: 0.5009504217598435 test: 0.9907384407934424


In [26]:
print (metrics.classification_report(X_tf_test.iloc[:,-1],y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.98      0.99     14221
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

    accuracy                           0.98     14221
   macro avg       0.20      0.20      0.20     14221
weighted avg       1.00      0.98      0.99     14221



Word2vec:

In [27]:
class mean_vectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(w2v.values())))

    def fit(self, X):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [28]:
odel = word2vec.Word2Vec(df.iloc[:,0],size=300,window=3, workers=5)
w2v = dict(zip(odel.wv.index2word, odel.wv.syn0))
data_mean=mean_vectorizer(w2v).fit(df.iloc[:,0]).transform(df.iloc[:,0])
df_w2v=pd.DataFrame(data=data_mean)
df_w2v['labels']=df.iloc[:,-1].tolist()
df_w2v.index=df.index
# df_w2v.to_csv('w2v.csv')
df_w2v.shape

(51305, 301)

In [29]:
# Data split
index_train=df.index<=split_ind
a=(index_train+0).sum()
# bow split
X_w2v_train, X_w2v_test =df_w2v.iloc[:a],df_w2v.iloc[a:]
del df_w2v

In [30]:
_,y_pred=my_log(X_w2v_train.iloc[:,:-1],X_w2v_test.iloc[:,:-1],X_w2v_train.iloc[:,-1],X_w2v_test.iloc[:,-1]) 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1620
           1       0.23      0.00      0.01      6670
           2       0.51      0.99      0.67     18924
           3       0.18      0.01      0.01      7750
           4       0.00      0.00      0.00      2120

    accuracy                           0.51     37084
   macro avg       0.18      0.20      0.14     37084
weighted avg       0.34      0.51      0.35     37084

train_macro: 0.1390296067164321 train_weighted: 0.34777196196932597 test: 0.9930253142149054


In [31]:
print (metrics.classification_report(X_w2v_test.iloc[:,-1],y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       1.00      0.99      0.99     14221
           3       0.00      0.00      0.00         0

    accuracy                           0.99     14221
   macro avg       0.33      0.33      0.33     14221
weighted avg       1.00      0.99      0.99     14221



In [32]:
# balance cut
for i in range(1,4):
    a=X_w2v_train['labels']==i
    a=X_w2v_train.index[a]
    shuffle_ap=np.random.permutation(len(a))
    index=a[shuffle_ap[3000:]]
    if i == 1:
        X_w2v_train1=X_w2v_train.drop(index=index)
    else:
        X_w2v_train1=X_w2v_train1.drop(index=index)

In [33]:
_,y_pred=my_log(X_w2v_train1.iloc[:,:-1],X_w2v_test.iloc[:,:-1],X_w2v_train1.iloc[:,-1],X_w2v_test.iloc[:,-1]) 

              precision    recall  f1-score   support

           0       0.19      0.01      0.02      1620
           1       0.27      0.46      0.34      3000
           2       0.39      0.48      0.43      3000
           3       0.28      0.32      0.30      3000
           4       0.23      0.05      0.08      2120

    accuracy                           0.30     12740
   macro avg       0.27      0.26      0.23     12740
weighted avg       0.28      0.30      0.27     12740

train_macro: 0.23259224833776493 train_weighted: 0.2665798073407488 test: 0.49878602343502587


In [34]:
print (metrics.classification_report(X_w2v_test.iloc[:,-1],y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.33      0.50     14221
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

    accuracy                           0.33     14221
   macro avg       0.20      0.07      0.10     14221
weighted avg       1.00      0.33      0.50     14221



In [35]:
_,y_pred=my_rf(X_w2v_train.iloc[:,:-1],X_w2v_test.iloc[:,:-1],X_w2v_train.iloc[:,-1],X_w2v_test.iloc[:,-1],200,150)  

train_macro: 0.9993182490348606 train_weighted: 0.9992179472696351 test: 0.9244441083043413


In [36]:
print (metrics.classification_report(X_w2v_test.iloc[:,-1],y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       1.00      0.86      0.92     14221
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

    accuracy                           0.86     14221
   macro avg       0.25      0.21      0.23     14221
weighted avg       1.00      0.86      0.92     14221



## СNN aka stupid expirement 

I am going to implement CNN model with custom naive and stupid custom loss function to try to work with class unbalance. However, for this experiment I do not use basic test set, only old school( take random from hole set). As a metric I chosed f1 macro.

In [114]:
# import torch
import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import copy

In [217]:
# define 2 leyers CNN
class mynet(nn.Module):
  def __init__(self, D_in, H, D_out):
    super(mynet, self).__init__()
    self.linear1 = nn.Linear(D_in, H) 
    self.linear2 = nn.Linear(H, D_out)
  def forward(self, x):
    inputs=self.linear1(x)
    act=nn.ReLU()
    x=act(inputs)
    x=self.linear2(x)
    return x

For penaly for my stupid lost function, I will try to not use weights sum like L2 or L1, instead I will check percentage of classes, multiply it to 10^-n. Probaply this approach will not work, but I will try...

In [83]:
# define batch split
def batch_split(X,y,batch_size=64):
    batch_indices = torch.LongTensor(np.random.permutation(X.shape[0]))
    limit=int(X.shape[0]/batch_size)
    batch_xs = torch.index_select(X,0,batch_indices[:limit])
    batch_ys = torch.index_select(y,0,batch_indices[:limit])
    return batch_xs,batch_ys

In [229]:
# define custom loss
def my_stupid_loss(outputs, labels,penalties):
    batch_size = outputs.shape[0]
    #define penalty
    k=penalties.mul(labels).sum()
    print ('loss:',k.numpy())
    # compute the log of softmax values
    outputs = F.log_softmax(outputs, dim=1) 
    mask=labels.byte()
    # pick the values corresponding to the labels
    outputs = outputs[mask]
    # calculate loss
    outputs=-((outputs.sum()+torch.log(k))/batch_size)
    return outputs

In [151]:
# define train function
def fit(X_train,y_train,X_val,y_val,batch_size,pen,epoch):
    best_val_scores=0
    best_model = copy.deepcopy(model.state_dict())
    for i in range(0,epoch):
        #split to batches
        batch_x,batch_y=batch_split(X_train,y_train,batch_size)
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(batch_x)
        # Compute and print loss
        loss = my_stupid_loss(y_pred,batch_y,pen)
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
        final=nn.Softmax()
        #epoch score
        y_pred=final(model(X_train))
        train_scores=metrics.f1_score(y_train.data.numpy().argmax(1),y_pred.data.numpy().argmax(1),average='macro')
        #batch train score
        y_pred=final(model(batch_x))
        batch_scores=metrics.f1_score(batch_y.data.numpy().argmax(1),y_pred.data.numpy().argmax(1),average='macro')
        #valid score
        y_pred=final(model(X_val))
        val_scores=metrics.f1_score(y_val.data.numpy().argmax(1),y_pred.data.numpy().argmax(1),average='macro')
        #Show res
        print('Train F1: {:.4f} Batch F1: {:.4f}  Val F1: {:.4f}'.format(train_scores, batch_scores, val_scores))
        #save best model
        if best_val_scores>val_scores:
            best_val_scores=val_scores
            best_model=copy.deepcopy(model.state_dict())
        model.load_state_dict(best_model)
    return model

## Load dataset check point

In [103]:
df=load_set('data_merge_clean.csv')

In [104]:
class mean_vectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(w2v.values())))

    def fit(self, X):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [105]:
# transform data
odel = word2vec.Word2Vec(df.iloc[:,0],size=300,window=3, workers=5)
w2v = dict(zip(odel.wv.index2word, odel.wv.syn0))
X=mean_vectorizer(w2v).fit(df.iloc[:,0]).transform(df.iloc[:,0])
y=df.iloc[:,-1].to_numpy()
X.shape

/home/anis_anton_v/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  This is separate from the ipykernel package so we can avoid doing imports until


(51305, 300)

In [107]:
# One hot labels endcoding
one=np.zeros((data_mean.shape[0],5))
for i in range(0,len(y)):
    one[i,y[i]]=1
y=one

In [108]:
# check shapes
X.shape[0]==y.shape[0]

True

## Data split

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=20)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=20)

In [110]:
X_train=Variable(torch.FloatTensor(X_train))
X_test=Variable(torch.FloatTensor(X_test))
X_val=Variable(torch.FloatTensor(X_val))
y_train=Variable(torch.FloatTensor(y_train))
y_test=Variable(torch.FloatTensor(y_test))
y_val=Variable(torch.FloatTensor(y_val))

## Start experiment

In [326]:
# initialize model
model=mynet(X_train.shape[1],220,5)

In [322]:
# balance percentage
pen=df['features'].value_counts().sort_index()/df['features'].value_counts().sum()
pen=Variable(torch.FloatTensor(pen))
pen*=1e-3

In [323]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [327]:
model=fit(X_train,y_train,X_val,y_val,batch_size=64,pen=pen,epoch=10)

loss: 0.1674341
Train F1: 0.1620 Batch F1: 0.1568  Val F1: 0.1658
loss: 0.16754822
Train F1: 0.1620 Batch F1: 0.1417  Val F1: 0.1658
loss: 0.16505079
Train F1: 0.1620 Batch F1: 0.1583  Val F1: 0.1658
loss: 0.16340503


/home/anis_anton_v/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/anis_anton_v/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/anis_anton_v/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Train F1: 0.1620 Batch F1: 0.1595  Val F1: 0.1658
loss: 0.17071506
Train F1: 0.1620 Batch F1: 0.1547  Val F1: 0.1658
loss: 0.16672148
Train F1: 0.1620 Batch F1: 0.1504  Val F1: 0.1658
loss: 0.16162227
Train F1: 0.1620 Batch F1: 0.1682  Val F1: 0.1658
loss: 0.16442269
Train F1: 0.1620 Batch F1: 0.1655  Val F1: 0.1658
loss: 0.16405791
Train F1: 0.1620 Batch F1: 0.1671  Val F1: 0.1658
loss: 0.16456789
Train F1: 0.1620 Batch F1: 0.1609  Val F1: 0.1658


In [328]:
final=nn.Softmax()
y_pred=final(model(X_test))
print (metrics.classification_report(y_test.data.numpy().argmax(1),y_pred.data.numpy().argmax(1)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       509
           1       0.14      0.47      0.21      2237
           2       0.66      0.57      0.61     10891
           3       0.00      0.00      0.00      2580
           4       0.00      0.00      0.00       714

    accuracy                           0.43     16931
   macro avg       0.16      0.21      0.16     16931
weighted avg       0.44      0.43      0.42     16931



/home/anis_anton_v/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


# I did several experiments and all of them shown that I did a terrible(stupid and wrong) shit. Moreover, I'm sure that I used W2v wrong.
 ## So anyway lets compare result:</br>
     1)The best model,based on assumption that test set is keggle test set and metrics f1 score, was RF tfidf and Log-reg w2v.
    BUT it is because test set consists only of targets from 2-nd class.
     2) In practice best F1 macro score on train set was shown by BOW Log reg.
 ## Summize, I am sure I did something wrong(maybe even all the task) ...